In [ ]:
using FFTW

In [ ]:
struct LaplaceOperator
    Lx:: Float64
    Ly:: Float64
    Nx:: Int
    Ny:: Int
    Δx:: Float64
    Δy:: Float64
    ncells:: Int
    kx²:: Vector{Float64}
    ky²:: Vector{Float64}
    function LaplaceOperator(Lx, Ly, Nx, Ny)
        Δx = Lx / Nx
        Δy = Ly / Ny
        ncells = Nx * Ny
        kx² = [(2 * sin(π * n / Nx) / Δx)^2 for n = 0:(Nx - 1)]
        ky² = [(2 * sin(π * n / Ny) / Δy)^2 for n = 0:(Ny - 1)]
        new(Lx, Ly, Nx, Ny, Δx, Δy, ncells, kx², ky²)
    end
end

In [ ]:
function Base.:*(Δ:: LaplaceOperator, x)
    x_arr = fft(reshape(x, Δ.Nx, Δ.Ny))
    for nx = 1:Δ.Nx
        for ny = 1:Δ.Ny
            x_arr[nx, ny] *= -(Δ.kx²[nx] + Δ.ky²[ny])
        end
    end
    y_arr = real.(ifft(x_arr))
    reshape(y_arr, Δ.ncells)
end

In [ ]:
struct LaplaceOperatorRef
    Lx:: Float64
    Ly:: Float64
    Nx:: Int
    Ny:: Int
    Δx:: Float64
    Δy:: Float64
    ncells:: Int
    function LaplaceOperatorRef(Lx, Ly, Nx, Ny)
        Δx = Lx / Nx
        Δy = Ly / Ny
        ncells = Nx * Ny
        new(Lx, Ly, Nx, Ny, Δx, Δy, ncells)
    end
end


In [ ]:
function Base.:*(Δ:: LaplaceOperatorRef, x)
    x_arr = reshape(x, Δ.Nx, Δ.Ny)
    y = similar(x)
    y_arr = reshape(y, Δ.Nx, Δ.Ny)
    for j₀ ∈ 1:Δ.Ny
        j₋₁ = j₀ == 1 ? Δ.Ny : j₀ - 1
        j₊₁ = j₀ == Δ.Ny ? 1 : j₀ + 1
        for i₀ ∈ 1:Δ.Nx
            i₋₁ = i₀ == 1 ? Δ.Nx : i₀ - 1
            i₊₁ = i₀ == Δ.Nx ? 1 : i₀ + 1
            y_arr[i₀, j₀] = ((x_arr[i₊₁, j₀] - 2 * x_arr[i₀, j₀] + x_arr[i₋₁, j₀]) / Δ.Δx^2
                + (x_arr[i₀, j₊₁] - 2 * x_arr[i₀, j₀] + x_arr[i₀, j₋₁]) / Δ.Δy^2)
        end
    end
    y
end

In [ ]:
Δ = LaplaceOperator(1.0, 1.5, 64, 128)
Δ_ref = LaplaceOperatorRef(Δ.Lx, Δ.Ly, Δ.Nx, Δ.Ny)

ux = [sin(2π * (n + 0.5) / Δ.Nx) for n = 0:(Δ.Nx-1)]
uy = [sin(4π * (n + 0.5) / Δ.Ny) for n = 0:(Δ.Ny-1)]
u = ux .* uy'
u_vec = reshape(u, Δ.ncells)

Δu = reshape(Δ * u_vec, Δ.Nx, Δ.Ny)
Δu_ref = reshape(Δ_ref * u_vec, Δ.Nx, Δ.Ny)

In [ ]:
@assert all(isapprox.(Δu, Δu_ref, atol=1e-15, rtol=1e-12))